# **「PyTorch入門 6. 最適化」**

## **パラメータの最適化**(参数的最优化)

在每次迭代中，计算model的输出，求出损失，以及对于每个参数关于损失的偏微分。

之后，利用梯度下降法做参数的最优化



## **コード準備**(代码的准备)

In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# 下载FashionMNIST数据集
# 训练用数据集
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
# 测试用数据集
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# 加载数据
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 构建神经网络
class NeuralNetwork(nn.Module):

    # 定义__init__()构造函数
    def __init__(self):

        # 调用父类的构造函数，确保父类nn.Module的初始化也被执行
        super(NeuralNetwork, self).__init__()

        # 定义了一个扁平化层，将输入的28x28图像转换为784维的一维向量
        # 第0维度代表样本编号，该维度不会通过nn.Flatten发生变化
        self.flatten = nn.Flatten()


        # 定义了一系列线性层和ReLU激活函数组成的顺序容器（sequential container）
        self.linear_relu_stack = nn.Sequential(

            # 第一个线性层，将输入的784维向量映射到512维
            nn.Linear(28*28, 512),
            # 一个ReLU激活函数，应用非线性变换
            nn.ReLU(),

            # 第二个线性层，将512维向量映射到另一个512维向量
            nn.Linear(512, 512),
            # 再次应用nn.ReLU()
            nn.ReLU(),

            # 第三个线性层，将512维向量映射到10维向量
            nn.Linear(512, 10),
            # 再应用一次nn.ReLU()非线性变换。
            nn.ReLU()
        )

    def forward(self, x):

        # 扁平化输入
        x = self.flatten(x)

        # 将扁平化后的向量通过定义的线性和ReLU层的堆栈
        logits = self.linear_relu_stack(x)

        # 返回最终的输出
        return logits

# 创建实例
model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:01<00:00, 13336851.87it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 209165.27it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3878381.10it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 14075799.86it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## **ハイパーパラメータ**(超参数 Hyperparameter)

在机器学习中,超参(数英语:Hyperparameter)是事先给定的,用来控制学习过程的参数。而其他参数(例如节点权重)的值是通过训练得出的.

因超参数值的不同,影响model的学习以及收束率.

本次实验中,使用的超参数如下.
* Number of Epochs: 迭代次数
* Batch Size: 构成mini-batch的数据数
* Learning Rate:参数更新的系数。值越小变化越小、过大时可能会导致训练失败

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## **最適化ループ**(最优化循环)

设定好超参数后,通过训练进行最优化循环,对model进行最优化

最优化的每一次迭代称为一个エポック(纪元 epoch)

每个纪元由两种循环构成

* 训练循环: 针对dataset进行训练,使得参数收束
* 验证/测试循环: 通过test dataset对moddel进行评价,确认是否性能有所提高

**損失関数：Loss Function**

即便是给定data,未经训练的 network 可能无法输出正确答案。

loss function是测定model推断的结果和实际正解之间的误差大小的函数

利用不停的训练,使loss function的值变小

为了计算损失,需要获取model对输入数据的推断,并比较该值与正确标签之间的差异

一般情况下,对于回归问题使用``nn.MSELoss``(Mean Square Error),分类问题使用``nn.NLLLoss``(Negative Log Likelihood`)

``nn.CrossEntropyLoss``是``nn.LogSoftmax`` 和 ``nn.NLLLoss``的结合

将model的输出``logit``値输入``nn.CrossEntropyLoss``进行正規化(标准化 Normalization),计算预测误差

In [4]:
# 定义loss function
loss_fn = nn.CrossEntropyLoss()

最优化器: Optimizer
为了使得各个训练步骤中model的误差变小,对model参数进行调整的进程

最优化算法: Optimization algorithms

实现最优化的具体步骤

最优化的逻辑都在``optimize``内

将model的参数,并把学习率作为超参数输入输入optimizer内进行初始化

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练过程中,最优化(optimization)由三个步骤构成

[1]``optimizer.zero_grad()`` 将model的梯度重置
梯度计算会不断积累,每次迭代都要重置

[2]``loss.backwards()`` 向后传播
PyTorch对损失计算每个参数的梯度

[3]``optimizer.step()``对各个参数的梯度进行超参数调整

## **実装全体:Full Implementation**(实现)

最优化实现的代码在``train_loop``中,使用test dataset对model性能评估的代码在``test_loop``中


In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    # 迭代遍历数据
    # X是输入数据,y是标签
    for batch, (X, y) in enumerate(dataloader):
        # 预测值与损失的计算
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 每进行100批次,打印一次当前损失和训练进度
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    # 禁止梯度计算
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            # 累加损失率
            test_loss += loss_fn(pred, y).item()
            # 累加正确率
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # pred.argmax(1) 获取每个样本的预测类别
            # == y 比较预测类别与实际类别，结果是一个布尔张量，然后将其转换为浮点型并求和。

    # 计算平均损失率
    test_loss /= size
    # 计算平均正确率
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
# 声明loss function
loss_fn = nn.CrossEntropyLoss()
# 最优化
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 迭代的次数(纪元数)
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    # 训练
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # 测试
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 1.307315  [    0/60000]
loss: 1.310146  [ 6400/60000]
loss: 1.168489  [12800/60000]
loss: 1.209316  [19200/60000]
loss: 1.285666  [25600/60000]
loss: 1.405125  [32000/60000]
loss: 1.298121  [38400/60000]
loss: 1.470510  [44800/60000]
loss: 1.216217  [51200/60000]
loss: 1.224859  [57600/60000]
Test Error: 
 Accuracy: 60.6%, Avg loss: 0.019474 

Epoch 2
-------------------------------
loss: 1.265004  [    0/60000]
loss: 1.276382  [ 6400/60000]
loss: 1.130377  [12800/60000]
loss: 1.175241  [19200/60000]
loss: 1.256235  [25600/60000]
loss: 1.374448  [32000/60000]
loss: 1.267017  [38400/60000]
loss: 1.449112  [44800/60000]
loss: 1.189181  [51200/60000]
loss: 1.200372  [57600/60000]
Test Error: 
 Accuracy: 61.2%, Avg loss: 0.019046 

Epoch 3
-------------------------------
loss: 1.227385  [    0/60000]
loss: 1.246226  [ 6400/60000]
loss: 1.097373  [12800/60000]
loss: 1.146831  [19200/60000]
loss: 1.232689  [25600/60000]
loss: 1.347103  [32000/600